### Task 2 -  Data Modeling and Transformation (Transform)

In [3]:
import os
import asyncio
import psycopg2
from telethon import TelegramClient
from telethon.tl.types import MessageMediaPhoto
from dotenv import load_dotenv
import sys
sys.path.append(os.path.abspath("../Script"))
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir("..") 
# print(os.getcwd())

In [4]:
from data_scraper import *

ModuleNotFoundError: No module named 'data_scraper'

In [3]:
# Load environment variables
load_dotenv(dotenv_path="../.env")  # adjust the path if needed
# Access the variables
api_id = os.getenv("tg_api_id")
api_hash = os.getenv("tg_api_hash")
session = os.getenv("session_name")
# PostgreSQL connection
def get_pgsql_connection():
    try:
        conn = psycopg2.connect(
          host="localhost",
          port=5432,
          database="telegram",
          user="postgres",
          password="Admin#123"
        )
        return conn
    except Exception as e:
        print(f"PostgreSQL connection error: {e}")
        return None

# Async image scraping function
async def scrape_images_async(channel_username, limit=50):
    client = TelegramClient(session_name, api_id, api_hash)
    await client.start()
    print(f"Connected to Telegram. Scraping images from {channel_username}...")

    conn = get_pgsql_connection()
    if conn is None:
        return
    cur = conn.cursor()

    save_dir = os.path.join("images", channel_username.strip("@"))
    os.makedirs(save_dir, exist_ok=True)

    try:
        async for message in client.iter_messages(channel_username,
limit=limit):
            if message.media and isinstance(message.media, MessageMediaPhoto):
                try:
                    file_name = f"{message.id}.jpg"
                    file_path = os.path.join(save_dir, file_name)
                    await client.download_media(message, file_path)

                    cur.execute("""
                        INSERT INTO images (message_id, channel_name,
file_name, file_path, posted_at)
                        VALUES (%s, %s, %s, %s, %s)
                        ON CONFLICT DO NOTHING
                    """, (
                        message.id,
                        channel_username,
                        file_name,
                        file_path,
                        message.date
                    ))

                    print(f"Saved image: {file_name}")
                except Exception as insert_err:
                    print(f"Error inserting {message.id}: {insert_err}")

        conn.commit()
    except Exception as scrape_err:
        print(f"Scraping failed: {scrape_err}")
    finally:
        cur.close()
        conn.close()
        await client.disconnect()
        print("Disconnected from Telegram.")